<a href="https://colab.research.google.com/drive/1oRHJGDS9lT1MteXfwHa3UpXhixwNOGle?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>